# Sujet 2: Modélisation des interdépendances

**Quel est l’impact d’une baisse de la capacité de LNG sur le système en général et en particulier sur
le système électrique ?**

Pour aller plus loin : modéliser le stockage gaz

**Authors**: Lucas DEFRANCE, Quentin DEHAENE, Jules PAINDESSOUS--POETE

## Imports

In [17]:
# Necessary imports
using JuMP
using XLSX
using DataFrames
#use the solver you want
using HiGHS
using CSV

In [18]:
# Define global variables
data_file_zootopia = "data/TP_empilement.xlsx";
data_file = "data/Donnees_etude_de_cas_ETE305.xlsx";

In [19]:
# Extract data for power mix
data_ranges = ["A2:A24", "D2:D24", "E2:E24", "F2:F24", "G2:G24", "H2:H24"]
column_names = [:plant_name, :nb_units, :pmax_per_unit, :pmin_per_unit, :dmin, :costs]
data_types = [String, Int, Float64, Float64, Int, Float64]

# Read data from file
df = DataFrame()
for (col_name, range, data_type) in zip(column_names, data_ranges, data_types)
    df[!, col_name] = data_type.(vec(XLSX.readdata(data_file, "Parc électrique", range)))
end

# Generate the plant_type vector using array comprehension
plant_type = [
    i in [14, 15] ? "thermal_fatal" :
    i == 18 ? "hydro_fatal" :
    i == 19 ? "hydro_lake" :
    i == 20 ? "hydro_step" :
    i in [21, 22] ? "wind" :
    i == 23 ? "solar" :
    "thermal" for i in 1:size(df, 1)
]

# Append the vector as a new column to the DataFrame
df[!, :plant_type] = plant_type

println(df)


23×7 DataFrame
 Row │ plant_name    nb_units  pmax_per_unit  pmin_per_unit  dmin   costs    plant_type    
     │ String        Int64     Float64        Float64        Int64  Float64  String        
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │ Iconuc               2          900.0          300.0     24     12.0  thermal
   2 │ Tabarnuc             2          900.0          300.0     24     12.0  thermal
   3 │ NucPlusUltra         2         1300.0          330.0     24     12.0  thermal
   4 │  Gazby               1          390.0          135.0      4     40.0  thermal
   5 │  Pégaz               2          394.0          137.0      4     40.0  thermal
   6 │  Samagaz             1          430.0          151.0      4     40.0  thermal
   7 │  Omaïgaz             2          430.0          151.0      4     40.0  thermal
   8 │  Gastafiore          1          430.0          151.0      4     40.0  thermal
   9 │ Igaznodon            2

In [20]:
# Initialize arrays to store values for non-fatal thermal clusters
names_th, costs_th, Pmin_th, Pmax_th, dmin_th = [], [], [], [], []

# Filter the DataFrame based on the plant_type column
df_thermal = filter(row -> row.plant_type == "thermal", df)

# Loop through each row of the DataFrame
for row in eachrow(df_thermal)
    # Iterate over each plant name and unit number in the row
    for i in 1:row.nb_units
        # Construct the complete plant name with unit number and append it to names_th
        push!(names_th, "$(row.plant_name)-$i")
        
        # Append other values to respective arrays
        push!(costs_th, row.costs)
        push!(Pmin_th, row.pmin_per_unit)
        push!(Pmax_th, row.pmax_per_unit)
        push!(dmin_th, row.dmin)
    end
end

# Convert the vectors to 2D arrays
costs_th, Pmin_th, Pmax_th, dmin_th = hcat(costs_th), hcat(Pmin_th), hcat(Pmax_th), hcat(dmin_th)

# Create a dictionary to map indices to plant names
Nth = length(costs_th)
dict_th = Dict(i => names_th[i] for i in 1:length(names_th))

# Check the value associated with index 1 in dict_th
dict_th[1]


"Iconuc-1"

In [21]:
#data for hydro reservoir
Nhy = 1 #number of hydro generation units
Pmin_hy = zeros(Nhy);
Pmax_hy = df[19,:pmax_per_unit] *ones(Nhy); #MW
e_hy = XLSX.readdata(data_file, "Détails historique hydro", "B2:B13");
# Calculate the mean stock level
weekly_mean_e_hy = repeat([sum(e_hy) / 52], Nhy)
costs_hy = df[19,:costs]*ones(Nhy); #MWh

#data for STEP
Pmax_STEP = df[20,:pmax_per_unit] #MW
rSTEP = 0.75
max_stock_STEP = 201600; #MWh

In [22]:
# We choose to use run 52 simulations of 1 week each
Tp = 24*7

168

In [ ]:
nb_week_year = 52
UC_continu, stock_STEP_continu, UP_continu, DO_continu = [], [], [], []
for w in 1:nb_week_year
    # We start the simulation at Tmin and end it at Tmax
    Tmin = (w-1)*24*7
    Tmax = Tmin + 24*7-1; #optimization by one week intervals
    Tp = 24*9;

        
    
    
    #data for load and fatal generation
    jour=XLSX.readdata(data_file_zootopia, "TP", "A$(10+Tmin):B$(Tp+Tmin + 10)");
    ToD=XLSX.readdata(data_file_zootopia, "TP", "B$(10+Tmin):B$(Tp+Tmin + 10)");
    load = XLSX.readdata(data_file_zootopia, "TP", "C$(10+Tmin):C$(Tp+Tmin + 10)");
    wind = XLSX.readdata(data_file_zootopia, "TP", "D$(10+Tmin):D$(Tp+Tmin + 10)");
    solar = XLSX.readdata(data_file_zootopia, "TP", "E$(10+Tmin):E$(Tp+Tmin + 10)");
    hydro_fatal = XLSX.readdata(data_file_zootopia, "TP", "F$(10+Tmin):F$(Tp+Tmin + 10)");
    thermal_fatal = XLSX.readdata(data_file_zootopia, "TP", "G$(10+Tmin):G$(Tp+Tmin + 10)");
    #total of RES
    Pres = wind + solar + hydro_fatal + thermal_fatal;
    # println(jour)
    
    #costs
    cth = repeat(costs_th', Tp) #cost of thermal generation €/MWh
    chy = repeat(costs_hy', Tp) #cost of hydro generation €/MWh
    cuns = 5000*ones(Tp) #cost of unsupplied energy €/MWh
    cexc = 0*ones(Tp); #cost of in excess energy €/MWh


    #############################
    #create the optimization model
    #############################
    model = Model(HiGHS.Optimizer)

    #############################
    #define the variables
    #############################
    #thermal generation variables
    
    @variable(model, Pth[1:Tp,1:Nth] >= 0)
    @variable(model, UCth[1:Tp,1:Nth], Bin)
    @variable(model, UPth[1:Tp,1:Nth], Bin)
    @variable(model, DOth[1:Tp,1:Nth], Bin)
    
    # contraintes de limitation de variation de la puissance (350 MWh/h 200 MWh CCG)
    @constraint(model, rampenuc1[t in 2:Tp, i in 1:6], Pth[t,i] - Pth[t-1,i] <= 350)
    @constraint(model, rampeccg[t in 2:Tp,i in 7:13], Pth[t,i] - Pth[t-1,i] <= 200)
    
    #hydro generation variables
    @variable(model, Phy[1:Tp,1:Nhy] >= 0)
    #unsupplied energy variables
    @variable(model, Puns[1:Tp] >= 0)
    #in excess energy variables
    @variable(model, Pexc[1:Tp] >= 0)
    #weekly STEP variables
    @variable(model, Pcharge_STEP[1:Tp] >= 0)
    @variable(model, Pdecharge_STEP[1:Tp] >= 0)
    @variable(model, stock_STEP[1:Tp] >= 0)


    #############################
    #define the objective function
    #############################
    @objective(model, Min, sum(Pth.*cth)+sum(Phy.*chy)+Puns'cuns+Pexc'cexc)


    #############################
    #define the constraints
    #############################
    #balance constraint
    @constraint(model, balance[t in 1:Tp], sum(Pth[t,g] for g in 1:Nth) + sum(Phy[t,h] for h in 1:Nhy) + Pres[t] + Puns[t] - load[t] - Pexc[t] - Pcharge_STEP[t] + Pdecharge_STEP[t]== 0)
    #thermal unit Pmax constraints
    @constraint(model, max_th[t in 1:Tp, g in 1:Nth], Pth[t,g] <= Pmax_th[g]*UCth[t,g])
    #thermal unit Pmin constraints
    @constraint(model, min_th[t in 1:Tp, g in 1:Nth], Pmin_th[g]*UCth[t,g] <= Pth[t,g])


    #thermal unit Dmin constraints
    for g in 1:Nth
        if (dmin_th[g] > 1)
            @constraint(model, [t in 2:Tp], UCth[t,g]-UCth[t-1,g]==UPth[t,g]-DOth[t,g],  base_name = "fct_th_$g")
            @constraint(model, [t in 1:Tp], UPth[t]+DOth[t]<=1,  base_name = "UPDOth_$g")
            if (w==1)
                @constraint(model, UPth[1,g]==0,  base_name = "iniUPth_$g")
                @constraint(model, DOth[1,g]==0,  base_name = "iniDOth_$g")
            else
                @constraint(model, UCth[1,g]-UC_continu[g]==UPth[1,g]-DOth[1,g],  base_name = "fct_th_0$g")
            end   
            @constraint(model, [t in dmin_th[g]:Tp], UCth[t,g] >= sum(UPth[i,g] for i in (t-dmin_th[g]+1):t),  base_name = "dminUPth_$g")
            @constraint(model, [t in dmin_th[g]:Tp], UCth[t,g] <= 1 - sum(DOth[i,g] for i in (t-dmin_th[g]+1):t),  base_name = "dminDOth_$g")
            if (w==1)
                @constraint(model, [t in 1:dmin_th[g]-1], UCth[t,g] >= sum(UPth[i,g] for i in 1:t), base_name = "dminUPth_$(g)_init")
                @constraint(model, [t in 1:dmin_th[g]-1], UCth[t,g] <= 1-sum(DOth[i,g] for i in 1:t), base_name = "dminDOth_$(g)_init")

            else
                @constraint(model, [t in 1:dmin_th[g]-1], UCth[t,g] >= sum(UPth[i,g] for i in 1:t)+sum(UP_continu[j,g] for j in 25 + t - dmin_th[g]:25), base_name = "dminUPth_$(g)_init_cont")
                @constraint(model, [t in 1:dmin_th[g]-1], UCth[t,g] <= 1-sum(DOth[i,g] for i in 1:t)-sum(DO_continu[j,g] for j in 25 + t - dmin_th[g]:25), base_name = "dminDOth_$(g)_init_cont")
            end
        end
    end 

    
    #hydro unit constraints
    @constraint(model, unit_bounds_hy[t in 1:Tp, h in 1:Nhy], Pmin_hy[h] <= Phy[t,h] <= Pmax_hy[h])
    #hydro stock constraint
    @constraint(model, stock_bounds_hy[h in 1:Nhy], sum(Phy[t,h] for t in 1:Tp) <= weekly_mean_e_hy[h] )

    #weekly STEP
    @constraint(model, max_Pcharge_STEP[t in 1:Tp],Pcharge_STEP[t] <= Pmax_STEP)
    @constraint(model, max_Pdecharge_STEP[t in 1:Tp],Pdecharge_STEP[t] <= Pmax_STEP)

    @constraint(model, stock_STEP_max[t in 1:Tp], stock_STEP[t] <= max_stock_STEP)
    @constraint(model, stock_STEP_evo[t in 2:Tp],stock_STEP[t] == stock_STEP[t-1] + Pcharge_STEP[t]*rSTEP - Pdecharge_STEP[t])
    if (w==1)
        @constraint(model,stock_STEP[1] == 0.15*max_stock_STEP)
    else
        @constraint(model,stock_STEP[1] == stock_STEP_continu + Pcharge_STEP[1]*rSTEP - Pdecharge_STEP[1])
    end

    #no need to print the model when it is too big
    #solve the model
    optimize!(model)
    
    # Collecting values for each iteration
    UC_continu= value.(UCth[Tp-48, 1:Nth])
    stock_STEP_continu= value(stock_STEP[Tp-48])
    UP_continu= value.(UPth[Tp-72:Tp-48, 1:Nth])
    DO_continu= value.(DOth[Tp-72:Tp-48, 1:Nth])

    
    #------------------------------
    #Results
    # println("semaine :",w)
    @show termination_status(model)
    # @show objective_value(model)

    #exports results as csv file
    th_gen = value.(Pth)
    hy_gen = value.(Phy)
    STEP_charge = value.(Pcharge_STEP)
    STEP_decharge = value.(Pdecharge_STEP)

    # new file created
    file_name = string("results/resultsW",w,".csv")
    touch(file_name)

    # file handling in write mode
    f = open(file_name, "w")

    write(f, "Jour, Heure, Production fatal, Demande, Thermique restant,")
    for i in 1:length(dict_th)
        plant_name = dict_th[i]
        write(f, string(plant_name, ","))
    end
    write(f,"Hydro, STEP pompage, STEP turbinage\n")
    

    
    for t in 1:Tp-48
        write(f, "$(jour[t]),  $(ToD[t]),$(Pres[t]),  $(load[t]), $(load[t]-Pres[t]),")
        for g in 1:Nth
            write(f, "$(th_gen[t,g]), ")
        end
        for h in 1:Nhy
            write(f, "$(hy_gen[t,h]),")
        end
    
        write(f, "$(STEP_charge[t]), $(STEP_decharge[t])\n")
    end
    

    close(f)
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
27567 rows, 18323 cols, 145059 nonzeros
21433 rows, 15845 cols, 147603 nonzeros
21376 rows, 15832 cols, 148152 nonzeros

Solving MIP model with:
   21376 rows
   15832 cols (9802 binary, 0 integer, 0 implied int., 6030 continuous)
   148152 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.6s
 R       0       0         0   0.00%   10818991.38462  10818991.38462     0.00%        0      0      0      3103     1.3s

Solving report
  Status            Optimal
  Primal bound      10818991.3846
  Dual bound        10818991.3846
  Gap               0% (tolerance: 0.01%)
  Solution status  

termination_status(model) = MathOptInterface.OPTIMAL
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
27636 rows, 18360 cols, 145592 nonzeros
21382 rows, 15812 cols, 147208 nonzeros
21124 rows, 15685 cols, 144057 nonzeros

Solving MIP model with:
   21124 rows
   15685 cols (9637 binary, 0 integer, 0 implied int., 6048 continuous)
   144057 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.7s
 R       0       0         0   0.00%   12196123.38462  12196123.38462     0.00%        0      0      0      2857     1.3s

Solving report
  Status            Optimal
  Primal bound      12196123.3846
  Dual bound        12196123.3846
  Gap   

termination_status(model) = MathOptInterface.OPTIMAL
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
27636 rows, 18360 cols, 145592 nonzeros
21382 rows, 15812 cols, 147208 nonzeros
21124 rows, 15685 cols, 144057 nonzeros

Solving MIP model with:
   21124 rows
   15685 cols (9637 binary, 0 integer, 0 implied int., 6048 continuous)
   144057 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.7s
 R       0       0         0   0.00%   12384295.38462  12384295.38462     0.00%        0      0      0      2690     1.2s

Solving report
  Status            Optimal
  Primal bound      12384295.3846
  Dual bound        12384295.3846
  Gap   

termination_status(model) = MathOptInterface.OPTIMAL
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
27636 rows, 18360 cols, 145592 nonzeros
21382 rows, 15812 cols, 147208 nonzeros
21124 rows, 15685 cols, 144057 nonzeros

Solving MIP model with:
   21124 rows
   15685 cols (9637 binary, 0 integer, 0 implied int., 6048 continuous)
   144057 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.8s
 R       0       0         0   0.00%   17159080.15385  17163310.15385     0.02%        0      0      0      2343     1.3s

26.1% inactive integer columns, restarting
Model after restart has 16540 rows, 13087 cols (7039 bin., 0 int., 0 impl., 

In [ ]:
# using CSV
# using DataFrames

# # File paths
# file_paths = []
# for i in 1:52
#     file_name = "results/resultsW$i.csv"
#     push!(file_paths, file_name)
# end

# # Initialize an empty DataFrame to store combined data
# combined_df = DataFrame()

# # Loop through each file path
# for file_path in file_paths
#     # Read the CSV file
#     df = CSV.read(file_path, DataFrame)
    
#     # If it's not the first file, skip the header
#     if nrow(combined_df) > 0
#         df = df[2:end, :]
#     end
    
#     # Append the data to the combined DataFrame
#     append!(combined_df, df)
# end

# # Write the combined DataFrame to a new CSV file
# CSV.write("results/combined_results.csv", combined_df)

